In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
import cv2
from skimage.io import imread
from keras.models import load_model

In [6]:
ROOT_PATH = os.getcwd()
TRAIN_PATH = ROOT_PATH + '/facial_feeling_detection/data/train/'
TEST_PATH = ROOT_PATH + '/facial_feeling_detection/data/test/'

In [7]:
def read_data(path, im_size):
    X = []
    ids = []

    ##### CODE #####
    # Iterar sobre todo lo que haya en path
    for file in os.listdir(path):

        # Leer la imagen a color y aplicarle el resize
        image = imread(path + file)
        smallimage = cv2.resize(image, (im_size, im_size))

        # Guardo en X
        X.append(smallimage)
        ids.append(file.replace('.jpg', ''))
    return np.array(X), ids

X_test, ids = read_data(TEST_PATH, 48)

In [8]:
X_test = X_test / 255.

In [9]:
_model = load_model('facial_feeling_detection/acc.48_model.h5')

In [10]:
y_pred = _model.predict(X_test)

221/221 [==============================] - 8s 34ms/step


In [12]:
predictions = []

for pred in y_pred:
    predictions.append(np.argmax(pred))

In [13]:
names = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
submission = pd.DataFrame(predictions, columns=['label']).replace(names)

In [15]:
submission['id_img'] = ids
submission = submission.loc[:, ::-1]
submission['id_img'] = submission['id_img'].astype(int)

In [18]:
test_set = pd.read_csv('test_set.csv')

In [19]:
submission_df = pd.merge(test_set, submission)

In [20]:
submission_df

,id_img,label
0,10052,sad
1,10065,surprise
2,10079,neutral
3,10095,sad
4,10121,sad
...,...,...
7061,9806,surprise
7062,9830,sad
7063,9853,surprise
7064,9878,fear


In [21]:
submission_df.to_csv('submission.csv', index=False)